In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import json
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
import collections
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import itertools
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.model_selection import GridSearchCV
import datetime, time
import pytz
import statsmodels.api as stats_api
import pickle as pickle_
import csv

/Users/satya/Anaconda/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# 1. Popularity Prediction

#### Initialize variables

In [3]:
hashtags = ['#gopatriots','#nfl','#sb49','#gohawks','#patriots','#superbowl'] #using only one tag
#hashtags = ['#sb49','#gohawks','#patriots','#superbowl'] #using only one tag
df_map = {} # to store dfs for each file

#### Reading files into dataframes and storing

In [ ]:
import os
df_columns = ['timestamp','tweet_id','author_id','followers','retweets','title','location']
bypass = False
for hashtag in hashtags:
    df = pd.DataFrame(columns=df_columns)
    lno = 0
    print("Processing ",hashtag)
    with open("tweet_data/tweets_"+hashtag+".txt", "r") as file_obj:
        if(not os.path.isfile('temp/'+hashtag+'.csv') or bypass):
            writer=csv.writer(open('temp/'+hashtag+'.csv','w'))
            writer.writerow(df_columns)
            line = file_obj.readline()
            while(line):
                if(lno%20000 == 0):
                    print(lno)
                j = json.loads(line)
                lno+=1
                timestamp =j['citation_date']
                followers =j['author']['followers']
                retweets = j['metrics']['citations']['total']
                author_id = j['author']['url']
                tweet_id = j['tweet']['id']
                title = j['title']
                location=j['tweet']['user']['location']
                writer.writerow([timestamp,tweet_id,author_id,followers,retweets,title,location])
                line = file_obj.readline()
        else:
            print("File Exists!")
            #writer.close()

Processing  #gopatriots
0
20000
Processing  #nfl
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
Processing  #sb49
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000
580000
600000
620000
640000
660000
680000
700000
720000
740000
760000
780000
800000
820000
Processing  #gohawks
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
Processing  #patriots
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
Processing  #superbowl
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000
580000
600000
620000
640000
660000
680000
700000
720000
740000
760000
780000
80

In [5]:
df_map = {}

In [5]:
for hashtag in hashtags:
    print('loaded dataframe',hashtag)
    df_map[hashtag] = pd.read_csv('temp/'+hashtag+'.csv')


loaded dataframe #gopatriots
loaded dataframe #nfl
loaded dataframe #sb49
loaded dataframe #gohawks
loaded dataframe #patriots
loaded dataframe #superbowl


## 1.1 Statistics

In [7]:
for hashtag in hashtags:
    print(hashtag)
    df = df_map[hashtag]
    print(df.count())

#gopatriots
timestamp    26232
tweet_id     26232
author_id    26232
followers    26232
retweets     26232
dtype: int64
#nfl
timestamp    259024
tweet_id     259024
author_id    259024
followers    259024
retweets     259024
dtype: int64
#sb49
timestamp    826951
tweet_id     826951
author_id    826951
followers    826951
retweets     826951
dtype: int64
#gohawks
timestamp    188136
tweet_id     188136
author_id    188136
followers    188136
retweets     188136
dtype: int64
#patriots
timestamp    489713
tweet_id     489713
author_id    489713
followers    489713
retweets     489713
dtype: int64
#superbowl
timestamp    1348762
tweet_id     1348762
author_id    1348762
followers    1348762
retweets     1348762
dtype: int64


In [ ]:
tweets_hrly = {}
retweets_hrly = {}
followers_hrly = {}
tweets_cnt = {}
retweets_cnt = {}
followers_cnt = {}

for hashtag in hashtags:
    print(hashtag)
    print(1)
    df = df_map[hashtag]
    print(2)
    sec = df['timestamp'].max() - df['timestamp'].min()   
    print(3)
    tweets_cnt[hashtag] = df['tweet_id'].count()
    print(4)
    retweets_cnt[hashtag] = df['retweets'].sum()
    print(5)
    followers_cnt[hashtag] = df['followers'].sum()
    print(6)
    tweets_hrly[hashtag] = (tweets_cnt[hashtag]*3600)/sec
    print(7)
    retweets_hrly[hashtag] = (retweets_cnt[hashtag]*3600)/sec
    print(8)
    followers_hrly[hashtag] = (followers_cnt[hashtag]*3600)/sec    


In [ ]:
print('Tweets Count : ',tweets_cnt)
print('Tweets Avg. hourly',tweets_hrly)
print('Retweets Count : ',retweets_cnt)
print('Retweets Avg. hourly',retweets_hrly)

print('Followers Count : ',followers_cnt)
print('Followers Avg. hourly',followers_cnt)

## 1.2 Linear Regression

#### K-fold cross validation from our previous project

In [ ]:
def perform_10fold(X,y,regressor, print_ = False,shuffle_= True):
    kf = KFold(n_splits=10,shuffle=shuffle_, random_state=0)
    i = 1
    bestModel = None
    tr_e = 0
    ts_e = 0
    min_ts_e = 10
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        regressor.fit(X_train,y_train)
        train_preds = regressor.predict(X_train)
        test_preds = regressor.predict(X_test)
        test_error = mean_squared_error(y_test,test_preds)
        tr_e += mean_squared_error(y_train,train_preds)
        ts_e += test_error
        if(print_):
            print("Fold : ",i)
            print("Training RMSE : ",np.sqrt(mean_squared_error(y_train,train_preds)))
            print("Test RMSE : ",np.sqrt(test_error))
        if(test_error<min_ts_e):
            min_ts_e = test_error
            bestModel = regressor
        i = i+1
    return np.sqrt(tr_e/10),np.sqrt(ts_e/10), bestModel

#### Returns the hour number from timestamp

In [ ]:
pst_tz = pytz.timezone('US/Mountain')
def getHourofDay(timestamp):
    #print(timestamp)
    return datetime.datetime.fromtimestamp(timestamp, pst_tz).hour

    
def getHour(timestamp):
    return int(timestamp/3600)

In [ ]:
X_map = {}
Y_map = {}
for hashtag in hashtags:
    print(hashtag)
    df = df_map[hashtag].copy()
    df['timestamp'] = df['timestamp'].apply(lambda x:getHour(x) )
    grouped = df.groupby('timestamp').agg({'followers': ['sum', 'max'],'tweet_id':'count', 'retweets':'sum'})
    grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
    #grouped.columns=grouped.columns.droplevel()
    grouped.reset_index(inplace=True)
    grouped['hour_of_day','tweet'] = grouped.apply(lambda x: getHourofDay(x['timestamp']*3600), axis=1)
    X_map[hashtag] = grouped.drop(['timestamp','tweet_id_count'],axis=1)
    Y_map[hashtag] = grouped['tweet_id_count']
    #print(grouped)
    

In [ ]:
for hashtag in hashtags:
    print(hashtag)
    model = stats_api.OLS(Y_map['#gopatriots'],X_map['#gopatriots']).fit()
    print(model.summary())
    